Let's train a model on MNIST handwritten digit dataset.

In [12]:
from torchvision import datasets, transforms

# Transform PIL image to Tensor 
# It includes normalization pixel value to range of [0, 1]
transform = transforms.Compose([transforms.ToTensor()])

# MNIST dataset
train_dataset = datasets.MNIST(root="./", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./', train=False, transform=transform, download=True)

Processing...
Done!


In [3]:
import matplotlib.pyplot as plt
plt.imshow(sample_img.squeeze(0).numpy() * 255, cmap='gray')
plt.show()

In [4]:
from torch.utils.data import DataLoader
# load the dataset into dataloader
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=16,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=16,
                         shuffle=False)

In [13]:
# define our model
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 10),
        )
        self.output = nn.Softmax(dim=1)
    
    def forward(self, x):
        # You can modify your model connection whatever you like
        out = self.fc(x.view(-1, 28*28))
        out = self.output(out)
        return out

In [24]:
import torch
device = torch.device('cpu')     # torch.device('cpu') for cpu

In [31]:
from torch.optim import Adam
model = Model()
model.to(device)
optimizer = Adam(model.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()


In [32]:
import numpy as np
# start training
model.train()

for epoch in range(10):
    train_loss = []
    train_acc = []
    for _, (img, target) in enumerate(train_loader):
        img_cuda = img.to(device)
        target_cuda = target.to(device)
        
        # You can also use
        # img_cuda = img.cuda()
        # target_cuda = target.cuda()
        
        
        optimizer.zero_grad()
        
        output = model(img_cuda)
        loss = loss_fn(output, target_cuda)
        loss.backward()
        optimizer.step()
        
        predict = torch.max(output, 1)[1]
        acc = np.mean((target_cuda == predict).numpy())
        
        train_acc.append(acc)
        train_loss.append(loss.item())
        
    print("Epoch: {}, Loss: {:.4f}, Acc: {:.4f}".format(epoch + 1, np.mean(train_loss), np.mean(train_acc)))

Epoch: 1, Loss: 1.7398, Acc: 0.7945
